In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
from sklearn.cluster import KMeans


import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv(r'C:\Users\martin.olivares\Desktop\projects\best-route\test_3.csv')
data.fillna(method='ffill', inplace=True)

df=pd.DataFrame()
df['address']=data['Direccion de inicio']
df['recogida']=data['Hora de recogida']

df.fillna(method='ffill', inplace=True)



In [3]:

#Función para verificar que las direcciones estén correctas. Posteriormente se dejan fuera las incorrectas (momentaneo)

def correct_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    if location:
        return location.address
    else:
        return 'no_existe_la_direccion'


df['corrected_address'] = df['address'].apply(correct_address)

filter=df['corrected_address']!='no_existe_la_direccion'
filter2= df['address']!='PASAJE CERO 14 CHORRILLOS, VIÑA DEL MAR'
df.where(filter & filter2,inplace=True)
df.dropna(inplace=True)


In [4]:
geolocator = Nominatim(user_agent="geoapiExercises")

grouped = df.groupby("recogida")

for name, group in grouped:

    # Calcula la distancia entre cada par de direcciones
    X = np.zeros((len(group), len(group)))
    for i in range(len(group)):
        address1 = group.iloc[i]['address']
        loc1 = geolocator.geocode(address1)
        lat1, lon1 = loc1.latitude, loc1.longitude
        point1 = (lat1, lon1)
        for j in range(i+1, len(group)):
            address2 = group.iloc[j]['address']
            loc2 = geolocator.geocode(address2)
            lat2, lon2 = loc2.latitude, loc2.longitude
            point2 = (lat2, lon2)
            X[i, j] = great_circle(point1, point2).m
            X[j, i] = great_circle(point1, point2).m

    # Crea una matriz con las distancias
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(X)
    group['label'] = kmeans.labels_

    # Continúa agrupando hasta que cada clúster tenga 4 o menos direcciones
    while group.groupby('label').agg({'address':'count'}).max().values[0] > 4:
        kmeans.n_clusters += 1
        kmeans.fit(X)
        group['label'] = kmeans.labels_
    df.loc[group.index, "label"] = group["label"]


In [5]:
df

,address,recogida,corrected_address,label
0,"DOCTOR ISRAEL ROIZBLATT 130, VALPARAISO",6:20,"Doctor Israel Roizblatt, Población Dr. Israel ...",0.0
1,"Ramaditas 1626, Valparaíso",6:20,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",0.0
2,"SEPTIMO DE LINEA 17, VALPARAÍSO",6:20,"Séptimo de Línea, Villa El Totoral, San Felipe...",0.0
3,"Dionisio Hernández 450,Viña del Mar",6:30,"Dionisio Hernández, Población Nuevo Horizonte,...",0.0
4,"Camila 109, Valparaíso",6:50,"109, Camila, Cerro San Juan de Dios, Almendral...",0.0
5,"Ramaditas 1626, Valparaíso",20:40,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",0.0
6,"DOCTOR ISRAEL ROIZBLATT 130, VALPARAISO",20:40,"Doctor Israel Roizblatt, Población Dr. Israel ...",1.0
7,"Navío San Martín 70, Valparaíso",20:40,"Navío San Martín, Bosque Inglés, San Roque, Va...",0.0
8,"Camila 109, Valparaíso",20:40,"109, Camila, Cerro San Juan de Dios, Almendral...",0.0
9,"Viana 1155, Valparaíso, Viña del Mar, Valparaíso",20:40,"Viana, Población Saenz, Forestal, Viña del Mar...",1.0
